# 🏥 Adaptive Reasoning for Healthcare Administration

## Real-World Multi-Task Reasoning with Symbolic AI

This demo showcases an **adaptive LLM-symbolic reasoning framework** solving realistic healthcare administration tasks, automatically routing each problem to the optimal solver.

---

### Scenario: Hospital Admin Assistant

A hospital administrator faces three different reasoning tasks in a single workflow:

<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin-top: 20px;">

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">Task 1: Clinical Trial Matching</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> Constraint Satisfaction</p>
<p style="font-size: 0.85em; opacity: 0.95;">Match patient profile against complex trial inclusion/exclusion criteria</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">Optimal solver: Z3 (SMT)</p>
</div>

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">Task 2: Policy Compliance</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> First-Order Logic</p>
<p style="font-size: 0.85em; opacity: 0.95;">Verify staff credentialing against hospital policies</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">Optimal solver: Prover9</p>
</div>

<div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">Task 3: Discharge Readiness</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> Logic Programming</p>
<p style="font-size: 0.85em; opacity: 0.95;">Evaluate patient discharge eligibility via forward chaining</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">Optimal solver: Pyke</p>
</div>

</div>

---

### Framework Capabilities

- **Automatic Problem Detection** - Identifies reasoning type without manual specification
- **Heterogeneous Solver Selection** - Routes to optimal symbolic solver (Z3, Prover9, Pyke)
- **Parallel Execution** - Processes independent tasks concurrently
- **End-to-End Automation** - From natural language to verified answers

## Step 1: Load Hospital Admin Tasks

Loading three realistic healthcare administration tasks that would typically be handled by different software systems.


In [11]:
problem_description = '''
PATIENT CASE FOR TRIAL MATCHING:
Patient Profile:
- 62-year-old male
- History: CVA, neurogenic bladder with indwelling suprapubic catheter
- Multiple prior admissions for UTIs, altered mental status, and urosepsis
- Current: Urosepsis due to monomicrobial multidrug-resistant E. coli (blood and urine cultures positive), now resolved after meropenem treatment
- CT findings: Non-obstructing stone in left ureter, no obstructive uropathy or urethral strictures
- Urinalysis: Positive for nitrites and leukocyte esterase, urine culture positive for MDR E. coli

Exclusions: No renal abscess, not a renal transplant recipient
Clinical Trial: NCT02142751
Trial Criteria:
Inclusion:
- ≥18 years old hospitalized patients
- Negative pregnancy test in fertile women
- Episode of monomicrobial urinary BSI due to multidrug-resistant E.coli (susceptible to fosfomycin and meropenem or ceftriaxone)
- Urinary sepsis with MDR E.coli from blood cultures, requires Urinalysis criteria (at least one): Positive dipstick for nitrites or leukocyte esterase, positive urine culture

Exclusion criteria:
- Polymicrobial bacteremia
- Undrained renal abscess or unresolved obstructive uropathy
- Pregnant or breastfeeding women
- Renal transplant recipients

Based on the available information, can this patient be enrolled in the trial?
A) Yes 
B) No

HOSPITAL CREDENTIALING VERIFICATION:
Excerpt from Hospital Policy Manual Section - Staff Credentialing Requirements:
- All hospital staff are classified as either clinical staff or administrative staff.
- All personnel who provide direct patient care are clinical staff.
- No clinical staff is credentialed if they have not completed annual infection control training.
- No clinical staff is credentialed if they have not completed annual HIPAA certification.
- No administrative staff provide direct patient care.
- All registered nurses provide direct patient care.

Personnel Record for Johnson, R.N.:
- Position: Registered Nurse
- Mandatory Compliance Status: Annual infection control training completed; Annual HIPAA certification not completed

Question: According to the system, Nurse Johnson is not credentialed. Is this correct?
A) Yes
B) No

DISCHARGE READINESS VERIFICATION:
Patient Discharge Protocol - Standard Operating Procedure:
- If a patient has stable vital signs, then the patient is medically stable.
- If a patient can perform self-care activities, then the patient is functionally independent.
- If a patient is medically stable and has no new symptoms, then the patient is ready for discharge evaluation.
- If a patient is functionally independent and has completed discharge teaching, then the patient is cleared for discharge.

Patient Status Report - Chen, Margaret (MRN: 445821):
- Margaret Chen has stable vital signs. 
- Margaret Chen can perform self-care activities. 
- Margaret Chen has completed discharge teaching. 
- Margaret Chen has no new symptoms.

Based on this discharge protocol, is Margaret Chen cleared for discharge?
A) Yes
B) No
'''

## Step 2: Devise Plan for Adaptive Reasoning 

The **Carnap Router** analyzes tasks and creates an execution plan.

Let's first initialize, load the config and initialize an LLM:

In [12]:
import yaml
from agents.generation.api import AzureOpenAIGenerator
from agents.meta_agents.planner import Planner, TracePersister
from helpers.trace_explainer import why
# Load config and initialize
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
azure_config = config['api_config']['gpt-4o-azure']
llm = AzureOpenAIGenerator(
    model_name=azure_config['model_name'],
    api_key=azure_config['api_key'],
    model_version=azure_config['openai_api_version'],
    azure_endpoint=azure_config['azure_endpoint']
)


Now, the router:

- Parses the multi-task input
- Classifies each task by reasoning type
- Constructs an execution DAG
- Routes tasks to appropriate symbolic solvers

In [13]:
router = Planner(generator=llm)
plans, memory, problem_ids = router({"problem": problem_description})
plan = plans[0]


2025-11-07 21:22:09,259 | INFO | Registered agent '<PLAN_START>'
2025-11-07 21:22:09,260 | INFO | Registered agent '<PLAN_END>'
2025-11-07 21:22:09,260 | INFO | Registered agent 'lp_solver'
2025-11-07 21:22:09,261 | INFO | Registered agent 'fol_solver'
2025-11-07 21:22:09,261 | INFO | Registered agent 'csp_solver'
2025-11-07 21:22:09,261 | INFO | Registered agent 'smt_solver'
2025-11-07 21:22:09,262 | INFO | Registered agent 'ilp_solver'
2025-11-07 21:22:09,262 | INFO | Registered agent 'epistemic_solver'
2025-11-07 21:22:09,262 | INFO | Registered agent 'risk_solver'
2025-11-07 21:22:09,263 | INFO | Registered agent 'compositional_solver'
2025-11-07 21:22:09,263 | INFO | Registered agent 'causal_solver'
2025-11-07 21:22:09,660 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 429 Too Many Requests"
2025-11-07 21:22:09,661 | INFO | Retrying request to /chat/completions in 16.000000 sec

## Step 3: Let CARNAP Solve It

We can now execute the plan!


In [14]:
trace_logger = TracePersister()
plan.execute(memory, trace_logger)


2025-11-07 21:22:31,591 | INFO | Execute plan with topological order...
2025-11-07 21:22:31,592 | INFO | Trace saved: <PLAN_START> → None
2025-11-07 21:22:31,592 | INFO | Trace saved: ques_3 → None
2025-11-07 21:22:31,593 | INFO | Trace saved: ques_2 → None
2025-11-07 21:22:31,593 | INFO | Trace saved: ques_1 → None
2025-11-07 21:22:34,068 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-07 21:22:34,073 | INFO | Scratchpad WRITE intermediate_form_lp_solver=Predicates:
stable_vitals($x, bool) ::: Does x have stable v (ttl=None)
2025-11-07 21:22:34,073 | INFO | Attempt 1/1
writing [compiled_krb]/facts.fbc
writing [compiled_krb]/rules_fc.py
writing [compiled_krb]/compiled_pyke_files.py
2025-11-07 21:22:34,103 | INFO | Scratchpad WRITE critique_outputs_lp_solver=True (ttl=None)
2025-11-07 21:22:34,103 | INFO | Pyke solver ran successfully.
2025-11-07 21:22:34,484 | INFO | 

Collecting results from memory:

In [15]:
predictions = []
for pid in problem_ids:
    result = memory.read(f"result_{pid}") or {}
    predictions.append(result.get('parsed_answer', None))

print(predictions)

['A', 'A', 'A']


## Step 4: Generate "Why" HTML

### Understanding Your Results

The output above shows **three predictions** from CARNAP—one for each healthcare task (clinical trial matching, staff credentialing, discharge readiness).

### Trace Analysis

Each prediction is backed by **generated symbolic code** visible in the trace above, showing the formal logic representations and solver-specific code (Z3, Prover9, or Pyke). However, manually examining traces is cumbersome.

### The `why()` Function

We've created a helper to make this effortless:

This uses an LLM call to format the trace and predictions into clean, interactive HTML that links each answer to its reasoning chain.

**Next step:** Run `why()` to see a beautiful explanation of how CARNAP arrived at each answer!

In [16]:
import importlib
import helpers.trace_explainer
importlib.reload(helpers.trace_explainer)
from helpers.trace_explainer import why
from IPython.display import HTML

task_titles = {
    "ques_1": "Clinical Trial Enrollment",
    "ques_2": "Credential Compliance",
    "ques_3": "Discharge Clearance",
}

html = why(
    llm,
    plan=plan,
    tracer=trace_logger,
    memory=memory,
    problem_statement=problem_description,
    output_path="demo_hospital_admin.html",
    model_args={"temperature": 0.15, "max_tokens": 6000},
    problem_ids=problem_ids,
    scenario="Hospital admin assistant",
    summary_fields=["parsed_answer", "raw_solver_output", "solver_name"],
    task_titles=task_titles,
    narrative_context="Carnap routed three hospital-admin subtasks (trial matching, credentialing, discharge) through the symbolic plan.",
    style_hint="Highlight the DAG, show per-task outcomes (A/B) as chips, and emphasise why each solver decision is trustworthy.",
)

2025-11-07 21:22:50,027 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 429 Too Many Requests"
2025-11-07 21:22:50,028 | INFO | Retrying request to /chat/completions in 41.000000 seconds
2025-11-07 21:23:36,828 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-07 21:23:37,022 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 429 Too Many Requests"
2025-11-07 21:23:37,023 | INFO | Retrying request to /chat/completions in 7.000000 seconds
2025-11-07 21:23:49,720 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-07 21:23:49,914 | INFO | HTTP Request: POST

In [17]:
HTML(html)

Agent,Goal
<PLAN_START>,"Special control marker, indicating the start of the whole working plan."
ques_3,Virtual agent representing ques_3
ques_2,Virtual agent representing ques_2
ques_1,Virtual agent representing ques_1
lp_solver,Determine whether a hypothesis can be derived from the given facts and rules using logical deduction..
fol_solver,"A theorem prover specialized for problems expressed in first-order logic (FOL). It is capable of handling complex reasoning tasks that involve quantifiers, predicates, and relationships among multiple entities."
smt_solver,A boolean satisfiability (SAT) solver designed to verify whether all logical constraints in a system are simultaneously satisfied. It is particularly effective for analytical reasoning tasks that involve checking whether a given configuration fulfills complex sets of conditions.
<PLAN_END>,"Special control marker, indicating the end of the whole working plan."
Assertion,Status
Age ≥ 18,Satisfied
